In [1]:
import pandas as pd
import glob
import os

# Get all CSV files in the current directory
csv_files = glob.glob('*.csv')
print(f"Found {len(csv_files)} CSV files:")
for file in csv_files:
    print(f"  - {file}")

# Read and combine all CSV files
dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    # Extract car number plate from filename (remove .csv extension and spaces)
    car_plate = file.replace('.csv', '').replace(' ', '')
    df['CarNumberPlate'] = car_plate
    dfs.append(df)
    print(f"Loaded {file}: {len(df)} rows -> CarNumberPlate: {car_plate}")

# Combine all dataframes
combined_df = pd.concat(dfs, ignore_index=True)

print(f"\nTotal rows in combined dataset: {len(combined_df)}")
print(f"Columns: {list(combined_df.columns)}")

Found 31 CSV files:
  - ABA 3768.csv
  - ABA 4603.csv
  - ABA 0048.csv
  - ABA 9519.csv
  - ABA 3745.csv
  - ABA 8248.csv
  - ABA 6852.csv
  - ABA 3043.csv
  - ABA 5354.csv
  - ABA 3243.csv
  - ABA 7031.csv
  - ABA 5945.csv
  - ABA 5005.csv
  - ABA 0948.csv
  - ABA 9007.csv
  - ABA 3715.csv
  - ABA 2209.csv
  - ABA 4889.csv
  - ABA 0557.csv
  - combined_data.csv
  - ABA 1845.csv
  - ABA 1474.csv
  - ABA 2654.csv
  - ABA 0692.csv
  - ABA 5927.csv
  - ABA 6593.csv
  - ABA 4342.csv
  - ABA 3836.csv
  - ABA 0319.csv
  - ABA 1198.csv
  - ABA 1371.csv
Loaded ABA 3768.csv: 67521 rows -> CarNumberPlate: ABA3768
Loaded ABA 4603.csv: 29092 rows -> CarNumberPlate: ABA4603
Loaded ABA 0048.csv: 44424 rows -> CarNumberPlate: ABA0048
Loaded ABA 9519.csv: 25953 rows -> CarNumberPlate: ABA9519
Loaded ABA 3745.csv: 35222 rows -> CarNumberPlate: ABA3745
Loaded ABA 8248.csv: 68942 rows -> CarNumberPlate: ABA8248
Loaded ABA 6852.csv: 70596 rows -> CarNumberPlate: ABA6852
Loaded ABA 3043.csv: 27790 rows -> 

/var/folders/04/wngkl13j3hj0s528zj5lpnj00000gn/T/ipykernel_91505/3544338673.py:14: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Loaded combined_data.csv: 2358738 rows -> CarNumberPlate: combined_data
Loaded ABA 1845.csv: 34276 rows -> CarNumberPlate: ABA1845
Loaded ABA 1474.csv: 57705 rows -> CarNumberPlate: ABA1474
Loaded ABA 2654.csv: 21480 rows -> CarNumberPlate: ABA2654
Loaded ABA 0692.csv: 57316 rows -> CarNumberPlate: ABA0692
Loaded ABA 5927.csv: 49842 rows -> CarNumberPlate: ABA5927
Loaded ABA 6593.csv: 11846 rows -> CarNumberPlate: ABA6593
Loaded ABA 4342.csv: 60264 rows -> CarNumberPlate: ABA4342
Loaded ABA 3836.csv: 38416 rows -> CarNumberPlate: ABA3836
Loaded ABA 0319.csv: 48568 rows -> CarNumberPlate: ABA0319
Loaded ABA 1198.csv: 15708 rows -> CarNumberPlate: ABA1198
Loaded ABA 1371.csv: 58726 rows -> CarNumberPlate: ABA1371

Total rows in combined dataset: 3538107
Columns: ['Timestamp', 'Latitude', 'Longitude', 'Speed', 'FuelLevelPercentage', 'FuelLevelLitre', 'EngineStatus', 'Direction', 'BatteryVoltage', 'Odometer', 'GPSLocated', 'CarNumberPlate', 'source_file']
Loaded ABA 3836.csv: 38416 rows ->

## Data Cleaning - Check Missing Values

In [2]:
# Check for missing values
print("=" * 60)
print("MISSING VALUES ANALYSIS")
print("=" * 60)

missing_summary = pd.DataFrame({
    'Column': combined_df.columns,
    'Missing_Count': combined_df.isnull().sum(),
    'Missing_Percentage': (combined_df.isnull().sum() / len(combined_df) * 100).round(2)
})

print(f"\nMissing values per column:")
print(missing_summary[missing_summary['Missing_Count'] > 0].to_string(index=False))

# Identify columns with high percentage of missing values
high_missing_threshold = 50  # 50% threshold
high_missing_cols = missing_summary[missing_summary['Missing_Percentage'] > high_missing_threshold]['Column'].tolist()

if high_missing_cols:
    print(f"\n⚠️  Columns with >{high_missing_threshold}% missing values (will be dropped):")
    for col in high_missing_cols:
        pct = missing_summary[missing_summary['Column'] == col]['Missing_Percentage'].values[0]
        print(f"  - {col}: {pct}%")
else:
    print(f"\n✓ No columns with >{high_missing_threshold}% missing values")

MISSING VALUES ANALYSIS

Missing values per column:
     Column  Missing_Count  Missing_Percentage
source_file        2358738               66.67

⚠️  Columns with >50% missing values (will be dropped):
  - source_file: 66.67%

Missing values per column:
     Column  Missing_Count  Missing_Percentage
source_file        2358738               66.67

⚠️  Columns with >50% missing values (will be dropped):
  - source_file: 66.67%


## Data Cleaning - Drop Columns and Duplicates

In [3]:
# Drop columns with >50% missing values
initial_shape = combined_df.shape
print("=" * 60)
print("DATA CLEANING STEPS")
print("=" * 60)

if high_missing_cols:
    combined_df = combined_df.drop(columns=high_missing_cols)
    print(f"\n✓ Dropped {len(high_missing_cols)} columns with >{high_missing_threshold}% missing values")
else:
    print(f"\n✓ No columns dropped (no columns with >{high_missing_threshold}% missing)")

# Check for duplicate rows
duplicates_count = combined_df.duplicated().sum()
print(f"\n📊 Duplicate rows found: {duplicates_count:,}")

if duplicates_count > 0:
    # Drop duplicates
    combined_df = combined_df.drop_duplicates()
    print(f"✓ Dropped {duplicates_count:,} duplicate rows")
else:
    print(f"✓ No duplicate rows to drop")

# Summary
print(f"\n{'BEFORE CLEANING':20} {initial_shape[0]:>10,} rows × {initial_shape[1]:>3} columns")
print(f"{'AFTER CLEANING':20} {combined_df.shape[0]:>10,} rows × {combined_df.shape[1]:>3} columns")
print(f"{'ROWS REMOVED':20} {initial_shape[0] - combined_df.shape[0]:>10,}")
print(f"{'COLUMNS REMOVED':20} {initial_shape[1] - combined_df.shape[1]:>10}")

DATA CLEANING STEPS

✓ Dropped 1 columns with >50% missing values

📊 Duplicate rows found: 1,187,259

📊 Duplicate rows found: 1,187,259
✓ Dropped 1,187,259 duplicate rows

BEFORE CLEANING       3,538,107 rows ×  13 columns
AFTER CLEANING        2,350,848 rows ×  12 columns
ROWS REMOVED          1,187,259
COLUMNS REMOVED               1
✓ Dropped 1,187,259 duplicate rows

BEFORE CLEANING       3,538,107 rows ×  13 columns
AFTER CLEANING        2,350,848 rows ×  12 columns
ROWS REMOVED          1,187,259
COLUMNS REMOVED               1


## Save Cleaned Combined Data

In [4]:
# Save to a new CSV file
output_file = 'combined_data.csv'
combined_df.to_csv(output_file, index=False)

print("=" * 60)
print(f"✓ Saved cleaned combined data to: {output_file}")
print("=" * 60)
print(f"  Total rows: {len(combined_df):,}")
print(f"  Total columns: {len(combined_df.columns)}")
print(f"  Columns: {list(combined_df.columns)}")
print(f"  Missing values remaining: {combined_df.isnull().sum().sum():,}")

# Display first few rows
print("\nFirst 5 rows of cleaned data:")
combined_df.head()

✓ Saved cleaned combined data to: combined_data.csv
  Total rows: 2,350,848
  Total columns: 12
  Columns: ['Timestamp', 'Latitude', 'Longitude', 'Speed', 'FuelLevelPercentage', 'FuelLevelLitre', 'EngineStatus', 'Direction', 'BatteryVoltage', 'Odometer', 'GPSLocated', 'CarNumberPlate']
  Missing values remaining: 0

First 5 rows of cleaned data:


,Timestamp,Latitude,Longitude,Speed,FuelLevelPercentage,FuelLevelLitre,EngineStatus,Direction,BatteryVoltage,Odometer,GPSLocated,CarNumberPlate
0,2025-08-01 00:01:05,3.48955,101.191783,0,25.46,24.187,False,269,25.42,846477.374,True,ABA3768
1,2025-08-01 00:03:05,3.48955,101.191783,0,25.46,24.187,False,269,25.41,846477.374,True,ABA3768
2,2025-08-01 00:05:04,3.48955,101.191783,0,25.46,24.187,False,269,25.41,846477.374,True,ABA3768
3,2025-08-01 00:07:05,3.48955,101.191783,0,25.46,24.187,False,269,25.41,846477.374,True,ABA3768
4,2025-08-01 00:09:05,3.48955,101.191783,0,25.46,24.187,False,269,25.41,846477.374,True,ABA3768
